### Step 1: Insert a project token and run the inserted code cell
<br/>A project token will allow you to access all the resources defined within this project.  By default, the token is inserted into the very first cell in a notebook.
<br/><img style="float: left;" src="https://github.com/yfphoon/dsx_local/blob/master/images/project_token.png?raw=true" alt="Project Token" />

<h1 id="tocheading">Attrition Demo</h1>
<div id="toc"></div>

<img src="https://github.com/CatherineCao2016/pics/raw/master/header.png" width="800" height="500" align="middle"/>

The Attrition demo focuses on retaining Merchants that are using company network for credit card processing. Here is the description of the case:

A client approved many low value merchant accounts without much scrutiny.  Many of those merchant accounts resulted in default. The client thinks that they should have put more of an emphasis on their applicant screening process. IBM suggests to enable fact based decision making for performance of its joint marketing programs.

This notebook will demostrate how to

1. Use Brunel and Seanborn library for visualizations

2. Use regular python Machine Learning libary scikit-learn and Spark's Machine Learning library(MLlib) for predicitive modeling in an intergrated environment on DSX.
3. Deploy SparkML model using Machine Learning Service

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

## Load Libraries

Run the following cell to import required libraries.

In [ ]:
import sklearn
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import pandas as pd
pd.options.display.max_columns = 999

import brunel

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import chi2_contingency,ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score

import numpy as np

import urllib3, requests, json

## Load Customer History Data

To load data into DSX environment using Object Storage, you need to:

1. Download the customer history data(CUST_HISTORY.csv) <a href="https://github.com/CatherineCao2016/AttritionDemo">here</a>.
2. Upload it into Object Storage by clicking the "1001" icon on the top right side -> Files -> Drag and drop or browse to select and upload the file.


In [ ]:
# 
# Access 'CUST_HISTORY.csv' data file from the project.
cust_spark = ProjectUtil.load_dataframe_from_file(pc, "CUST_HISTORY.csv")
cust_spark.show()


Create a Pandas DataFrame from the Spark DataFrame.  A Pandas DataFrame is required for the analysis below

In [ ]:
cust_pd=cust_spark.toPandas()
cust_pd.head()

### Dataset Overview

Let's take a quick look at the dataset.

In [ ]:
print "There are " + str(len(cust_pd)) + " observations in the customer history dataset."
print "There are " + str(len(cust_pd.columns)) + " variables in the dataset."

print "\n******************Descriptive statistics*****************************\n"
print cust_pd.describe()

print "\n******************Dataset Quick View*****************************\n"
cust_pd.head()

## Exploratory Data Analysis

In this section, we will explore the dataset further with some visualizations.

Two open source libraries are used:
* <a href="https://github.com/Brunel-Visualization/Brunel">Brunel</a> is a high-level language that describes visualizations in terms of composable actions. It drives a visualization engine (D3) that performs the actual rendering and interactivity. Brunel makes it much easier to build fun and inventive visualizations in Jupyter notebooks.

* <a href="https://github.com/Brunel-Visualization/Brunel">Seaborn</a> is a Python visualization library based on matplotlib. It provides a high-level interface for drawing attractive statistical graphics.

### Outcome Variable: Account Default

In [ ]:
%brunel data('cust_pd') x(IS_DEFAULT) y(#count) color(IS_DEFAULT) bar tooltip(#all)

As you can see from the bar chart, 300 out of 1000 accounts are in default.

### Default by Credit Program

In [ ]:
%brunel data('cust_pd') polar stack bar y(#count) color(CREDIT_PROGRAM) percent(#count) tooltip(#all) | stack bar x(CREDIT_PROGRAM) y(#count) color(IS_DEFAULT) bin(CREDIT_PROGRAM) percent(#count) label(#count) tooltip(#all) :: width=1200, height=350 

* Top 3 credit programs with most Merchants are Electronics(28%), New Car(23.4%) and Furniture(18.1%).
* Top 3 credit programs with high default rate are Education(44%), New Car(38%), and Retraining(35.1%)

### Default by IS_XBORDER

In [ ]:
%brunel data('cust_pd') polar stack bar y(#count) color(IS_XBORDER) percent(#count) tooltip(#all) | stack bar x(IS_XBORDER) y(#count) color(IS_DEFAULT) bin(IS_XBORDER) percent(#count) label(#count) tooltip(#all) :: width=1200, height=350 

Most Merchants have cross-border transactions. Relatively, they have a lower default rate than those don't have coross-border transactions.

### RENT vs. IS_DEFAULT

In [ ]:
%brunel data('cust_pd') stack bar x(RENT) y(#count) color(IS_DEFAULT: blue-red) bin(RENT) sort(RENT) percent(#count) label(#count) tooltip(#all)

From this stacked bar chart, we can see that Merchants who rent their residence have higher default rate.

### HISTORY vs. IS_DEFAULT

In [ ]:
%brunel data('cust_pd') bar x(HISTORY) y(#count) color(HISTORY) tooltip(#all) | stack bar x(HISTORY) y(#count) color(IS_DEFAULT: green-red) bin(HISTORY) sort(HISTORY) percent(#count) label(#count) tooltip(#all) :: width=1200, height=350 

### AMOUNT_K_USD vs. IS_DEFAULT

In [ ]:
sub_yes = cust_pd[cust_pd["IS_DEFAULT"] == "Yes"]
sub_no = cust_pd[cust_pd["IS_DEFAULT"] == "No"]
    
p_value = ttest_ind(sub_yes['AMOUNT_K_USD'], sub_no["AMOUNT_K_USD"], equal_var = False)[1]

fig, axs = plt.subplots(nrows= 1, figsize=(13, 5))
sns.boxplot(x = "IS_DEFAULT", y = "AMOUNT_K_USD", data = cust_pd, showfliers=False, palette="Set2")
if p_value < .05:
    plt.title("AMOUNT_K_USD" + "\n P value:" + str(p_value) + "\n The distributions for the two groups are significantly different!" + "\n Default: mean/std.: " + str(sub_yes["AMOUNT_K_USD"].describe()[1]) + "/" + str(sub_yes["AMOUNT_K_USD"].describe()[2]) + "\n Non-default: mean/std.: " + str(sub_no["AMOUNT_K_USD"].describe()[1]) + "/" + str(sub_no["AMOUNT_K_USD"].describe()[2]))
else:
    plt.title("AMOUNT_K_USD" + "\n P value:" + str(p_value) + "\n Default: mean/std.: " + str(sub_yes["AMOUNT_K_USD"].describe()[1]) + "/" + str(sub_yes["AMOUNT_K_USD"].describe()[2]) + "\n Non-default: mean/std.: " + str(sub_safe["AMOUNT_K_USD"].describe()[1]) + "/" + str(sub_no["AMOUNT_K_USD"].describe()[2]))           

In this box plot, the visualization is enhanced by T-test statistics. The result is significant which indicates that the average credit amount for the non-default group and default group are different. Default group has larger average credit amount.



### Default rate by state

In [ ]:
default_rate = pd.crosstab(cust_pd.IS_DEFAULT, cust_pd.STATE).apply(lambda r: r/r.sum(), axis=0)

default_rate2 = default_rate.T

%brunel data('default_rate2') map color(Yes) key(STATE) label(STATE)


Brunel also provides a very neat way for map visualization. So for this use case, all the Merchants come from 4 states: NY, NJ, PA and CT.

### Correlation Matrix

A heatmap is used to visualize the correlations between all continuous variables.

In [ ]:
plt.figure(figsize=(12, 8))

corr_df = cust_pd.iloc[:,1:].corr()

sns.heatmap(corr_df, 
            xticklabels = corr_df.columns.values,
            yticklabels = corr_df.columns.values,
            annot = True);


* There is no strong correlation between most variables.
* The correlation between AMOUNT_K_USD and CONTRACT_DURATION_MONTH is moderate.

## Modeling And Evaluation

For demo purpose, we will use RandomForestClassifier() from sklearn to rank feature importance, and Logistic Regression from Spark's Machine Learning Library(MLlib) for modeling.

### Sklearn Random Forest: Rank Feature Importance

In [ ]:
# convert IS_DEFAULT to 1/0
le = LabelEncoder()

cust_pd.loc[:,'IS_DEFAULT']= le.fit_transform(cust_pd.loc[:,'IS_DEFAULT'])

y = np.float32(cust_pd.IS_DEFAULT)
X = cust_pd.copy()

# drop y 
X = cust_pd.drop(['IS_DEFAULT', 'MERCHANT'], axis = 1)

# Prepocess the data: Encode categorical variables into numeric representations

categoricalColumns = ["ACCT_STATUS_K_USD", "BRANCHES",'HISTORY', 'CREDIT_PROGRAM', 'ACCOUNT_TYPE', 'ACCT_AGE', 'STATE', 'IS_URBAN', 'IS_XBORDER','SELF_REPORTED_ASMT', 'CO_APPLICANT', 'GUARANTOR','PRESENT_RESIDENT', 'OWN_REAL_ESTATE', 'PROP_UNKN','OTHER_INSTALL_PLAN', 'RENT', 'OWN_RESIDENCE','TELEPHONE', 'SHIP_INTERNATIONAL']

for col in categoricalColumns:
    X[col] = le.fit_transform(X[col])

# scale X
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

# split the data to training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


random_forest = RandomForestClassifier()

#Stratify split and train on 5 folds
skf = StratifiedKFold(y_train, n_folds=5)
counter = 1
for train_fold, test_fold in skf:
    random_forest.fit(X_train[train_fold], y_train[train_fold])
    
    print( str(counter) + ": ", random_forest.score(X_train[test_fold], y_train[test_fold]))
    counter += 1 
    
#### local notes: one interesting error here, if you don't do the import correctly, it will show error Params must be either a param map or a list/tuple of param maps, but got <class 'pandas.core.series.Series'>.

From the result of 5-fold CV, the average accuracy varies. The model is not very stable, one possible reason is that our sample size is really small. We will need to restrict the model complexity. We will choose top 10 important features for further analysis.

In [ ]:
features_order = cust_pd.drop(['IS_DEFAULT', 'MERCHANT'], axis = 1).columns.tolist()

feature_importance_dict = {key: val for key, val in zip(features_order, random_forest.feature_importances_)}

for k in sorted(feature_importance_dict, key=feature_importance_dict.get, reverse=True):
    print k, feature_importance_dict[k]

### Default Prediction: Spark MLlib Logistic Regression

In this section, We will use Logistic Regression from Spark MLlib to predict defalut.<br/>
We will use the Spark DataFrame for building the Spark ML model

In [ ]:
cust_spark.printSchema()

**Preprocess the data**

In [ ]:
# group top 10 features to categorical and numerical
allCols = cust_pd.drop(['MERCHANT','IS_DEFAULT'], 1).columns.tolist()
importantCols = ['AMOUNT_K_USD', 'ACCT_STATUS_K_USD', 'CONTRACT_DURATION_MONTH', 'ESTABLISHED_MONTH', 'HISTORY', 'CREDIT_PROGRAM', 'ACCT_AGE', 'ACCOUNT_TYPE', "PRESENT_RESIDENT", "STATE"]
importantCols_num = ['AMOUNT_K_USD', 'CONTRACT_DURATION_MONTH', 'ESTABLISHED_MONTH']
importantCols_cat = np.setdiff1d(importantCols, importantCols_num).tolist()

In [ ]:
# create label_str column
cust_spark = cust_spark.withColumnRenamed("IS_DEFAULT", 'label_str')

In [ ]:
# One-hot encoder for all categorical varaibles
for categoricalCol in importantCols_cat:
    cust_spark = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index").fit(cust_spark).transform(cust_spark)
    cust_spark = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec").transform(cust_spark)  

In [ ]:
# Assemble feature vector
assemblerInputs = map(lambda c: c + "classVec", importantCols_cat) + importantCols_num
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
cust_spark = assembler.transform(cust_spark)

In [ ]:
# Transform the label_str column to a numeric value
cust_spark = StringIndexer(inputCol='label_str', outputCol='label').fit(cust_spark).transform(cust_spark)

In [ ]:
# keep useful variables
selectedcols = ["label", "features"]
cust_model = cust_spark.select(selectedcols)

** Split the data into training and testing sets **

In [ ]:
trainingData, testData = cust_model.randomSplit([0.7, 0.3], seed = 824)
print trainingData.count()
print testData.count()

**Train Logistic Regression Model**

In [ ]:
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

** Use CrossValidator and ParamGridBuilder to search for best model **

In [ ]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [1.0,0.3,0.1, 0.03,0.01,0.0]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

evaluator = BinaryClassificationEvaluator()
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cvModel = cv.fit(trainingData)

** Use BinaryClassificationEvaluator to evaluate the model**

Note that the default metric for the BinaryClassificationEvaluator is *areaUnderROC*

A rough guide for classifying the accuracy of a test:

    .90-1 = excellent (A)
    .80-.90 = good (B)
    .70-.80 = fair (C)
    .60-.70 = poor (D)
    .50-.60 = fail (F)

So the model performance is fair.

In [ ]:
evaluator.evaluate(cvModel.transform(testData))

** Final Model**

We will move forward with the model with Top 10 features.

**Hyperparameters used in final model**

In [ ]:
print cvModel.bestModel._java_obj.getRegParam()
print cvModel.bestModel._java_obj.getMaxIter()
print cvModel.bestModel._java_obj.getElasticNetParam()
print cvModel.bestModel._java_obj.getThreshold()

** Intercept and Weights **

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
print "Model Intercept = " + str(cvModel.bestModel.intercept)

coefficients = cvModel.bestModel.coefficients
coefficients = map(lambda w: (float(w),), coefficients)
weightsDF = sqlContext.createDataFrame(coefficients, ["Feature Weight"])
weightsDF.show()

## Create Pipeline for ML

In [ ]:
cust_spark1 = sqlContext.createDataFrame(cust_pd).withColumnRenamed("IS_DEFAULT", 'label')

trainingData3, testData3 = cust_spark1.randomSplit([0.7, 0.3], seed = 824)

SI1 = StringIndexer(inputCol='ACCOUNT_TYPE', outputCol='ACCOUNT_TYPE'+"Index")
SI2 = StringIndexer(inputCol='ACCT_AGE', outputCol='ACCT_AGE'+"Index")
SI3 = StringIndexer(inputCol='ACCT_STATUS_K_USD', outputCol='ACCT_STATUS_K_USD'+"Index")
SI4 = StringIndexer(inputCol='CREDIT_PROGRAM', outputCol='CREDIT_PROGRAM'+"Index")
SI5 = StringIndexer(inputCol='HISTORY', outputCol='HISTORY'+"Index")
SI6 = StringIndexer(inputCol='PRESENT_RESIDENT', outputCol='PRESENT_RESIDENT'+"Index")
SI7 = StringIndexer(inputCol='STATE', outputCol='STATE'+"Index")


OH1 = OneHotEncoder(inputCol='ACCOUNT_TYPE'+"Index", outputCol='ACCOUNT_TYPE'+"classVec")
OH2 = OneHotEncoder(inputCol='ACCT_AGE'+"Index", outputCol='ACCT_AGE'+"classVec")
OH3 = OneHotEncoder(inputCol='ACCT_STATUS_K_USD'+"Index", outputCol='ACCT_STATUS_K_USD'+"classVec")
OH4 = OneHotEncoder(inputCol='CREDIT_PROGRAM'+"Index", outputCol='CREDIT_PROGRAM'+"classVec")
OH5 = OneHotEncoder(inputCol='HISTORY'+"Index", outputCol='HISTORY'+"classVec")
OH6 = OneHotEncoder(inputCol='PRESENT_RESIDENT'+"Index", outputCol='PRESENT_RESIDENT'+"classVec")
OH7 = OneHotEncoder(inputCol='STATE'+"Index", outputCol='STATE'+"classVec")

assembler_features = VectorAssembler(inputCols=['ACCOUNT_TYPEclassVec','ACCT_AGEclassVec','ACCT_STATUS_K_USDclassVec','CREDIT_PROGRAMclassVec','HISTORYclassVec','PRESENT_RESIDENTclassVec', 'STATEclassVec',
 'AMOUNT_K_USD',
 'CONTRACT_DURATION_MONTH',
 'ESTABLISHED_MONTH'], outputCol="features")

lr_final = LogisticRegression(maxIter=10, regParam=0.1, elasticNetParam=0.0, threshold = 0.5, labelCol="label", featuresCol="features")

pipeline_lr = Pipeline(stages=[SI1, SI2, SI3,SI4,SI5,SI6,SI7, OH1,OH2,OH3,OH4,OH5,OH6,OH7, assembler_features, lr_final])
model_lr = pipeline_lr.fit(trainingData3)

## Save Model in ML repository


In [ ]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

service_path = 'https://internal-nginx-svc.ibm-private-cloud.svc.cluster.local:12443'
ml_repository_client = MLRepositoryClient()

model_artifact = MLRepositoryArtifact(model_lr, training_data=trainingData3, name="Predict_CC_Default")

saved_model = ml_repository_client.models.save(model_artifact)

# Print the saved model properties
print "modelType: " + saved_model.meta.prop("modelType")
print "creationTime: " + str(saved_model.meta.prop("creationTime"))
print "modelVersionHref: " + saved_model.meta.prop("modelVersionHref")
print "label: " + saved_model.meta.prop("label")

## Deploy model via UI

Save the notebook and switch to the Analytic Assets tab of the project (hint: open with another tab in your browser).
Under Models, find and click into your deployed model. Add an Online deployment. 

## Summary

In the notebook, you 

* used Brunel and Seaborn library for data visualizations
* used regular python Machine Learning libary scikit-learn and Spark's Machine Learning library(MLlib) for predicitive modeling

This notebook has been developed by Catherine Cao (Catherine.Cao@ibm.com). Some of the code was contributed by Dustin Vanstee(vanstee@us.ibm.com) and Sidney Phoon(yfphoon@us.ibm.com).